In [1]:
import os
import sys
import csv
import glob
import pandas as pd
import numpy as np
import subprocess
from multiprocessing import Pool
from sklearn.utils import shuffle

In [17]:
#### TarBase_v8 Database
#### This is not be used now! 

In [ ]:
# def process_TarBase():
################################################################################
gff = "/home/galaxy/data/miRNA_area/hsa.gff3"
# a decade-long collection of experimentally supported miRNA–gene interactions
db_file = "/home/galaxy/data/miRNA_area/TarBase_v8/TarBase_v8_download.txt"
result_dir = "/home/galaxy/project/QTL_analysis/related_with_miRNA/eQTL"
################################################################################
df = pd.read_table(db_file, sep="\t")
print(set(df.category.tolist()))
included_list = [
    "Normal/Primary", "Embryonic/Fetal,Stem/Progenitor", "Embryonic/Fetal",
    "Pulmonary Artery", "Stem/Progenitor"
]
df_hs = df[(df.species == "Homo sapiens") & (~df.tissue.isnull()) &
           (df.category.isin(included_list))]
df_hs["name"] = df_hs["mirna"] + ":" + df_hs["geneId"]
df_hs["info"] = df_hs.iloc[:, 3:].apply(
    lambda x: ":".join(x.astype(str)), axis=1)
print(df_hs.head())
########
# process_TarBase()

df_gff = pd.read_table(gff, sep="\t", comment="#", header=None)
df_gff["mirna"] = df_gff.iloc[:, -1].str.split("Name=").str[1].str.split(
    ";").str[0]
df_gff_bed = df_gff.iloc[:, [0, 3, 4, -1, 5, 6]]
df_gff_bed.columns = ["chr", "start", "end", "mirna", "dot", "strand"]
print(df_gff_bed.head())

#####
df_raw = df_hs.merge(df_gff_bed, on="mirna", how="left").dropna()
df_region = df_raw[[
    "chr", "start", "end", "name", "dot", "strand", "info", "tissue"
]]
df_region["start"] = df_region["start"].astype("int")
df_region["end"] = df_region["end"].astype("int")
df_region = df_region.sort_values(["chr", "start",
                                   "end"]).reset_index(drop=True)
print(df_region.head())

#####
# df_region.to_csv(os.path.join(result_dir, "TarBase_GRCh38.bed"), sep="\t", header=False, index=False)
df_region = df_region[["chr", "start", "end"]].drop_duplicates()
tissue_list = list(set(df_region.tissue.tolist()))
for tissue in tissue_list:
    df_t = df_region[df_region.tissue == tissue]
    result_file = os.path.join("/home/galaxy/data/miRNA_area/TarBase_v8/",
                               "%s.bed" % tissue.strip())
    df_t.iloc[:, :6].to_csv(
        result_file,
        sep="\t",
        header=False,
        index=False,
        quoting=csv.QUOTE_NONE)

#### Split TarBase_GRCh38.bed by each miRNA

In [6]:
result_dir = "/home/galaxy/data/miRNA_area/TarBase_v8/split_by_mirna/info_dir"
source_file = "/home/galaxy/data/miRNA_area/TarBase_v8/TarBase_GRCh38.bed"
df = pd.read_table(source_file, sep="\t", header=None)
df.columns = ["chr", "start", "end", "name", "dot", "strand", "info", "tissue"]
df.head()

,chr,start,end,name,dot,strand,info,tissue
0,chr1,1167160,1167181,hsa-miR-200b-3p:ENSG00000001461,.,+,Homo sapiens:HT29:Intestine:Cancer/Malignant:M...,Intestine
1,chr1,1167160,1167181,hsa-miR-200b-3p:ENSG00000003096,.,+,Homo sapiens:HT29:Intestine:Cancer/Malignant:M...,Intestine
2,chr1,1167160,1167181,hsa-miR-200b-3p:ENSG00000003436,.,+,Homo sapiens:NCIH508:Intestine:Cancer/Malignan...,Intestine
3,chr1,1167160,1167181,hsa-miR-200b-3p:ENSG00000005189,.,+,Homo sapiens:HT29:Intestine:Cancer/Malignant:M...,Intestine
4,chr1,1167160,1167181,hsa-miR-200b-3p:ENSG00000005206,.,+,Homo sapiens:HT29:Intestine:Cancer/Malignant:M...,Intestine


In [8]:
df["miRNA"] = df["name"].str.split(":").str[0]
df["Gene"] = df["name"].str.split(":").str[1]

""


In [16]:
# df.groupby(["miRNA"]).apply(generate_info_file)
df.groupby(["miRNA"]).apply(generate_bed_file)

""


In [7]:
def generate_info_file(df_i):
    result_dir = "/home/galaxy/data/miRNA_area/TarBase_v8/split_by_mirna/info_dir"
    mirna_name = df_i["miRNA"].tolist()[0]
    result_file = os.path.join(result_dir, "%s.bed" % mirna_name)
    df_i.to_csv(result_file, sep="\t", index=False)

In [15]:
def generate_bed_file(df_i):
    result_dir = "/home/galaxy/data/miRNA_area/TarBase_v8/split_by_mirna/GRCh38_bed_dir"
    mirna_name = df_i["miRNA"].tolist()[0]
    result_file = os.path.join(result_dir, "%s.bed" % mirna_name)
    df_bed = df_i[["chr", "start", "end"]].sort_values(by=["chr", "start"]).drop_duplicates()
    df_bed.to_csv(result_file, sep="\t", header=False, index=False)